# Computes Fragments from the PDB Fragment File

First select which data set you would like to generate.

In [5]:
import ipywidgets as widgets
from glob import glob
from os.path import basename, splitext

datasets = [basename(x) for x in glob("Geometry/*")]
w = widgets.Dropdown(
    options=datasets,
    description='Data Set:',
    disabled=False,
)
display(w)

Dropdown(description=u'Data Set:', options=('TimeStep', 'Size'), value='TimeStep')

## Define the Input Files

In [15]:
namelist = []
dataset = w.value
for f in glob("Geometry/"+dataset+"/*.pdb"):
    namelist.append(splitext(basename(f))[0])

## Use the Babel Program To Convert PDB To Yaml

In [17]:
import subprocess
from BigDFT import Fragments
bab_wrap = Fragments.__file__.split("install")[0] + "bigdft/tests/libs/wrappers/babel"

prog = widgets.IntProgress(
    max=len(namelist),
    description='Loading:',
)
display(prog)

for f in namelist:
    infile = "Geometry/"+dataset+"/"+f+".pdb"
    outfile = infile.replace(".pdb", ".yaml")
    with open(outfile, "w") as ofile:
        subprocess.call([bab_wrap, "-i", infile], stdout=ofile)
    prog.value += 1
prog.bar_style="success"
prog.description="Finished"

IntProgress(value=0, description=u'Loading:', max=5)

## Read the input

In [35]:
from yaml import load
frag_indices = {}

for f in namelist:
    frag_indices[f] = []
    with open("Geometry/"+dataset+"/"+f+".yaml", 'r') as ifile:
        values = load(ifile)
        positions = values["Reading positions"].itervalues().next()
        frag_indices[f] = Fragments.CreateFragDict(positions)

## Convert To A List of Lists

In [41]:
frag_lists = {}
for f in namelist:
    new_list = []
    for key, value in frag_indices[f].items():
        for key2, value2 in frag_indices[f][key].items():
            new_list.append(value2)
    frag_lists[f] = new_list

## Write To Yaml File

In [42]:
for f in namelist:
    with open("FragmentInput/"+dataset+"/"+f+"-frag.yaml", 'w') as ofile:
        for value in frag_lists[f]:
            ofile.write("- "+str(value)+"\n")

IOError: [Errno 2] No such file or directory: 'FragmentInput/Size/1L2Y-3.0-frag.yaml'